In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
census=pd.read_csv('census_data.csv')

In [3]:
census.tail()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [4]:
census.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,10.080679,1077.648844,87.303830,40.437456
std,13.640433,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [5]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [6]:
census['income_bracket']=census['income_bracket'].apply(label_fix)

In [7]:
census.tail()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0
32560,52,Self-emp-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,1


In [8]:
X=census.drop(['income_bracket'],axis=1)

In [9]:
y=census['income_bracket']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [12]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [13]:
Age=tf.feature_column.numeric_column('age')
Education_num=tf.feature_column.numeric_column('education_num')
Capital_gain=tf.feature_column.numeric_column('capital_gain')
Capital_loss=tf.feature_column.numeric_column('capital_loss')

In [14]:
Work_class=tf.feature_column.categorical_column_with_hash_bucket('workclass',hash_bucket_size=1000)
Education=tf.feature_column.categorical_column_with_hash_bucket('education',hash_bucket_size=1000)
Marital_status=tf.feature_column.categorical_column_with_hash_bucket('marital_status',hash_bucket_size=1000)
Occupation=tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=1000)
Relation=tf.feature_column.categorical_column_with_hash_bucket('relationship',hash_bucket_size=1000)
Race=tf.feature_column.categorical_column_with_hash_bucket('race',hash_bucket_size=1000)
Gender=tf.feature_column.categorical_column_with_vocabulary_list('gender',['Male','Female'])
Country=tf.feature_column.categorical_column_with_hash_bucket('native_country',hash_bucket_size=1000)

In [15]:
feat_cols=[Age,Education,Education_num,Capital_gain,Capital_loss,Work_class,Marital_status,Occupation,Relation,Gender,Race,Country]

In [16]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,
                                              batch_size=10,
                                              num_epochs=1000,
                                              shuffle=True)

In [17]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols,
                                n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_session_config': None, '_model_dir': '/tmp/tmpeklfk5zx', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000}


In [18]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpeklfk5zx/model.ckpt.
INFO:tensorflow:step = 1, loss = 6.93147
INFO:tensorflow:global_step/sec: 266.173
INFO:tensorflow:step = 101, loss = 393.028 (0.377 sec)
INFO:tensorflow:global_step/sec: 306.655
INFO:tensorflow:step = 201, loss = 3.24973 (0.328 sec)
INFO:tensorflow:global_step/sec: 289.528
INFO:tensorflow:step = 301, loss = 77.072 (0.343 sec)
INFO:tensorflow:global_step/sec: 294.259
INFO:tensorflow:step = 401, loss = 4.46907 (0.342 sec)
INFO:tensorflow:global_step/sec: 286.031
INFO:tensorflow:step = 501, loss = 0.27751 (0.350 sec)
INFO:tensorflow:global_step/sec: 281.916
INFO:tensorflow:step = 601, loss = 2.3162 (0.355 sec)
INFO:tensorflow:global_step/sec: 305.417
INFO:tensorflow:step = 701, loss = 120.387 (0.325 sec)
INFO:tensorflow:global_step/sec: 283.729
INFO:tensorflow:step = 801, loss = 2.45983 (0.352 sec)
INFO:tensorflow:global_step/sec: 313.177
INFO:tensorflow:step = 901, loss 

In [19]:
pred_input_func=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [20]:
predictions=list(model.predict(input_fn=pred_input_func))

INFO:tensorflow:Restoring parameters from /tmp/tmpeklfk5zx/model.ckpt-5000


In [21]:
predictions[12]

{'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([ 0.02133423], dtype=float32),
 'logits': array([-3.82587719], dtype=float32),
 'probabilities': array([ 0.97866577,  0.02133423], dtype=float32)}

In [22]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [23]:
from sklearn.metrics import classification_report

In [24]:
print(classification_report(y_true=y_test,y_pred=final_preds))

             precision    recall  f1-score   support

          0       0.91      0.85      0.88      7428
          1       0.61      0.73      0.66      2341

avg / total       0.84      0.82      0.83      9769

